# Introduction to finite grammar

In [1]:
from estnltk import Text

text = Text('Veski 5, Elva, Tartumaa.').tag_layer(['words'])

In [2]:
from estnltk.taggers import AddressPartTagger

address_part_tagger = AddressPartTagger()
address_part_tagger.tag(text)
text.address_parts

Layer(name=address_parts, spans=SL[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(Tartumaa, {'grammar_symbol': 'MAAKOND', 'type': 'maakond'})]])

## Rule
reeglite tegemine

SEQ reeglid

In [3]:
from estnltk.finite_grammar import Rule, Grammar


Rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3)

ADDRESS -> TÄNAV MAJA ASULA	: 3, val: default_validator, dec: default_decorator, scoring: default_scoring

## Grammar

In [4]:
grammar = Grammar(start_symbols=['ADDRESS'], 
                  rules=None, # the default, deprecated
                  depth_limit=float('inf'), # the default
                  width_limit=float('inf'), # the default
                  legal_attributes=None # the default
                  )

grammar.add(Rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3))
grammar.add(Rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3))
grammar


Grammar:
	start: ADDRESS
	terminals: ASULA, MAJA, TÄNAV
	nonterminals: ADDRESS
	legal attributes: frozenset()
	depth_limit: inf
	width_limit: inf
Rules:
	ADDRESS -> TÄNAV MAJA ASULA	: 3, val: default_validator, dec: default_decorator, scoring: default_scoring
	ADDRESS -> TÄNAV MAJA	: 3, val: default_validator, dec: default_decorator, scoring: default_scoring

## GrammarParsingTagger

In [5]:
from estnltk.taggers import GrammarParsingTagger

tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol', # the default
                              layer_name='addresses_1', # default: 'parse'
                              attributes=(), # default: ()
                              output_ambiguous=True # default False, True recommended
                              )
tagger

GrammarParsingTagger(input_layers=['address_parts'], output_layer=addresses_1, output_attributes=(), grammar=
Grammar:
	start: ADDRESS
	terminals: ASULA, MAJA, TÄNAV
	nonterminals: ADDRESS
 ..., type: <class 'estnltk.finite_grammar.grammar.Grammar'>, name_attribute=grammar_symbol, output_nodes={'ADDRESS'}, resolve_support_conflicts=True, resolve_start_end_conflicts=True, resolve_terminals_conflicts=True, ambiguous=True, gap_validator=None)

In [6]:
tagger.tag(text)

Text(text="Veski 5, Elva, Tartumaa.")

In [7]:
text.addresses_1

Layer(name=addresses_1, spans=SL[AS[ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]]],
AS[ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})]]]])

# Priorities

In [8]:
grammar = Grammar(start_symbols=['ADDRESS'], 
                  rules=None, # the default, deprecated
                  depth_limit=float('inf'), # the default
                  width_limit=float('inf'), # the default
                  legal_attributes=None # the default
                  )

grammar.add(Rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=2))
grammar.add(Rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3))
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol', # the default
                              layer_name='addresses_2', # the default
                              attributes=(), # default: ()
                              output_ambiguous=True # default False
                              )
tagger.tag(text)
text.addresses_2

Layer(name=addresses_2, spans=SL[AS[ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})]]]])

# Decorators

In [9]:
def address_decorator(nodes):
    asula = ''
    maakond = ''
    t2nav = ''
    indeks = ''
    maja = ''
    for node in nodes:
        if node.name == 'ASULA':
            asula = node.text#[0]
        elif node.name == 'TÄNAV':
            t2nav = node.text#[0]
        elif node.name == 'MAAKOND':
            maakond = node.text#[0]
        elif node.name == 'MAJA':
            maja = node.text#[0]
        elif node.name == 'INDEKS':
            indeks = node.text#[0]
    return {'grammar_symbol': 'ADDRESS',
            'ASULA': asula,
            'TÄNAV': t2nav,
            'INDEKS': indeks,
            'MAAKOND': maakond,
            'MAJA': maja}

grammar = Grammar(start_symbols=['ADDRESS'], 
                  rules=None, # the default, deprecated
                  depth_limit=float('inf'), # the default
                  width_limit=float('inf'), # the default
                  legal_attributes=['INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA']
                  )

grammar.add(Rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3, decorator=address_decorator))
grammar.add(Rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3, decorator=address_decorator))
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol',
                              layer_name='addresses_3',
                              attributes=('INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA'),
                              output_nodes=None,
                              resolve_support_conflicts=True,
                              resolve_start_end_conflicts=True,
                              resolve_terminals_conflicts=True,
                              output_ambiguous=False # default False
                              )
tagger.tag(text)
text.addresses_3

Layer(name=addresses_3, spans=SL[ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]],
ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})]]])

# Validators

In [10]:
text = Text('Inimesed, kes töötavad Tartus Ülikooli 5, Elva haiglas \
            ja Tõravere observatooriumis, söövad esmaspäeviti õunu.').tag_layer(['words'])
address_part_tagger.tag(text)

grammar = Grammar(start_symbols=['ADDRESS'],
                  legal_attributes=['INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA']
                  )

grammar.add(Rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3, decorator=address_decorator))
grammar.add(Rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3, decorator=address_decorator))
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol',
                              layer_name='addresses_4',
                              attributes=('INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA')
                              )
tagger.tag(text)
text.addresses_4

Layer(name=addresses_4, spans=SL[ES[AS[Span(Ülikooli, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]],
ES[AS[Span(Ülikooli, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})]]])

In [11]:
text = Text('Inimesed, kes töötavad Tartus Ülikooli 5, Elva haiglas \
            ja Tõravere observatooriumis, söövad esmaspäeviti õunu.').tag_layer(['words'])
address_part_tagger.tag(text)

town_streets = {'Elva': {'Veski', 'Tuletõrje'},
                'Tartu': {'Veski', 'Ülikooli'}}

def validator(node):
    street = node[0].text
    town = node[2].text
    if town in town_streets:
        return street in town_streets[town]
    return True

grammar = Grammar(start_symbols=['ADDRESS'], 
                  legal_attributes=['INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA']
                  )

grammar.add(Rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3, decorator=address_decorator, validator=validator))
grammar.add(Rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3, decorator=address_decorator))
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol',
                              layer_name='addresses_4',
                              attributes=('INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA'),
                              output_ambiguous=True
                              )
tagger.tag(text)
text.addresses_4

Layer(name=addresses_4, spans=SL[AS[ES[AS[Span(Ülikooli, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]]]])

## `SEQ` rules

In [12]:
def address_decorator(nodes):
    asula = ''
    maakond = ''
    t2nav = ''
    indeks = ''
    maja = ''
    for node in nodes:
        if node.name == 'ASULA':
            asula = node.text#[0]
        elif node.name == 'TÄNAV':
            t2nav = node.text#[0]
        elif node.name == 'MAAKOND':
            maakond = node.text#[0]
        elif node.name == 'MAJA':
            maja = node.text#[0]
        elif node.name == 'SEQ(MAJA)':
            maja = [n.text for n in node.support]
        elif node.name == 'INDEKS':
            indeks = node.text#[0]
    return {'grammar_symbol': 'ADDRESS',
            'ASULA': asula,
            'TÄNAV': t2nav,
            'INDEKS': indeks,
            'MAAKOND': maakond,
            'MAJA': maja}

text = Text('Veekatkestus Tartu Veski tänava majades 3, 5, 7.').tag_layer(['words'])
address_part_tagger.tag(text)

grammar = Grammar(start_symbols=['ADDRESS'],
                  legal_attributes=['INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA']
                  )
def scoring(node):
    return len(node[2].support)

grammar.add(Rule('ADDRESS', 'ASULA TÄNAV SEQ(MAJA)', group='g0', priority=3, decorator=address_decorator, scoring=scoring))
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol',
                              layer_name='addresses_4',
                              attributes=('INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA'),
                              output_ambiguous=True
                              )
tagger.tag(text)
text.addresses_4

Layer(name=addresses_4, spans=SL[AS[ES[AS[Span(Tartu, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Tartu, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(3, {'grammar_symbol': 'MAJA', 'type': None})]]],
AS[ES[AS[Span(Tartu, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Tartu, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(3, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]]],
AS[ES[AS[Span(Tartu, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Tartu, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(3, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(7, {'grammar_symbol': 'MAJA', 'type': None})]]]])